In [125]:
from requests import Request, Session, get
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import time
import pandas as pd
from alpha_vantage.cryptocurrencies import CryptoCurrencies
apiKey = 'TPSIJ2LQA0IWGG0N'

In [84]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': "968c4aee-3bcb-41f3-890f-c947c70bdc61",
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [85]:
cryptos = []
for crypto in data['data']:
    coin = {
        "id" : crypto['id'],
        "name" : crypto['name'],
        'symbol' : crypto['symbol'],
        "price": crypto["quote"]["USD"]["price"],
        "market_cap": crypto["quote"]["USD"]["market_cap"],
        'slug' : crypto['slug'],
        'num_market_pairs' : crypto['num_market_pairs'],
        'date_added' : crypto['date_added'],
        'max_supply' : crypto['max_supply'],
        'circulating_supply' : crypto['circulating_supply'],
        'total_supply' : crypto['total_supply'],
        'platform' : crypto['circulating_supply'],
        'cmc_rank' : crypto['total_supply']
        
    }
    cryptos.append(coin)

In [110]:
df = (pd.DataFrame(cryptos).
      sort_values(
          by=["market_cap"],
          ascending=False
      ).
      head(300)[["name", "symbol", "market_cap"]]
     )

In [122]:
df.to_csv(r'/Users/d/desktop/crypto_data/mkt_caps.csv', index = False)
df

,name,symbol,market_cap
0,Bitcoin,BTC,8.049762e+11
1,Ethereum,ETH,3.785787e+11
2,Binance Coin,BNB,7.703843e+10
3,Cardano,ADA,6.504670e+10
4,Dogecoin,DOGE,6.229759e+10
...,...,...,...
296,Kleros,PNK,1.540165e+08
297,PARSIQ,PRQ,1.535201e+08
298,Wrapped NXM,WNXM,1.531042e+08
291,PAC Protocol,PAC,1.514221e+08


In [117]:
def get_crypto(crypto):
    cc = CryptoCurrencies(apiKey, output_format='pandas')
    df, metadata = cc.get_digital_currency_daily(symbol=crypto, market = "usd")
    df = df.drop(["1b. open (USD)", "2b. high (USD)", "3b. low (USD)", "4b. close (USD)"], axis=1)
    df.columns = ["Open", "High", "Low", "Close", "Volume", "Mkt Cap"]
    return df["Close"].iloc[::-1]

all_series = []
names = []

for symbol in df["symbol"].values:
    try:
        prices = get_crypto(symbol)
        all_series.append(pd.Series(data = prices, index = prices.index))
        names.append(symbol)
        time.sleep(12) #must make notebook wait since limited to 5 api calls per min
    except:
        continue

In [118]:
index = all_series[0].index
the_series = [x.reindex(index, fill_value = None) for x in all_series]
df2 = pd.concat(all_series, axis = 1)
df2.columns = names

In [126]:
df2.to_csv(r'/Users/d/desktop/crypto_data/pricing_data.csv', index = True)
df2

,BTC,ETH,BNB,ADA,DOGE,XRP,DOT,BCH,UNI,LTC,...,FIL,AAVE,XMR,WAVES,ZEC,EGLD,XEM,ENJ,BTG,ONT
date,,,,,,,,,,,,,,,,,,,,,
2018-08-22,6360.89,270.80,9.5888,0.08955,NaN,0.31850,NaN,NaN,NaN,55.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0800
2018-08-23,6525.01,275.99,9.7587,0.09232,NaN,0.32630,NaN,NaN,NaN,57.29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1810
2018-08-24,6681.64,281.19,10.2038,0.09381,NaN,0.32646,NaN,NaN,NaN,57.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1650
2018-08-25,6733.64,277.55,10.1058,0.09413,NaN,0.32702,NaN,NaN,NaN,57.89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.3460
2018-08-26,6700.00,274.29,10.2158,0.09306,NaN,0.32276,NaN,NaN,NaN,56.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.3850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-13,49670.97,3717.91,571.1000,1.92960,0.48945,1.36400,39.988,1258.52,37.345,317.95,...,117.82,509.33,402.34,31.874,296.36,161.31,0.3093,2.0556,93.739,2.1275
2021-05-14,49841.45,4075.01,602.6100,2.00690,0.55971,1.40000,47.979,1296.38,40.400,326.23,...,120.65,594.78,412.91,33.414,305.67,164.15,0.3194,2.1229,102.135,2.1795
2021-05-15,46762.99,3646.68,563.4100,2.17440,0.50855,1.49040,43.643,1171.75,36.607,300.30,...,110.48,517.89,382.38,31.189,271.08,156.79,0.2971,1.9524,95.189,1.9754


In [128]:
pwd

'/Users/d'